In [1]:
!python --version

Python 3.8.8


In [2]:
import pandas as pd
import glob
import numpy as np
import datetime

# Data Ingestion and ETL

## Ingest all name pattern matched files

In [3]:
all_dir = glob.glob("interactions*.csv")
list_df = []
for file in all_dir:
    df = pd.read_csv(file)
    list_df.append(df)

interactions = pd.concat(list_df, axis = 0)

In [4]:
print(interactions.columns)
print(interactions.dtypes)

Index(['interaction_time', 'user_id', 'item_id', 'interaction_type', 'price'], dtype='object')
interaction_time     object
user_id              object
item_id               int64
interaction_type     object
price               float64
dtype: object


In [5]:
print(interactions.groupby('user_id').count()['interaction_time'].mean())
interactions.groupby('user_id').count().sort_values(by = 'interaction_time')

12.622526623226898


,interaction_time,item_id,interaction_type,price
user_id,,,,
fffff770-2fd5-4873-a533,1,1,1,1
6ca6c725-a001-4f53-88d5,1,1,1,1
6ca6f79c-df39-493b-9c82,1,1,1,1
6ca73c86-a97d-4ec4-bdbe,1,1,1,1
d313031a-6bdf-41e9-bb46,1,1,1,1
...,...,...,...,...
635fcf0d-2aed-4459-98c1,8122,8122,8122,8122
b6064465-47cf-48e8-b4ae,8283,8283,8283,8283
cd9c16de-0949-424b-a90a,9420,9420,9420,9420


In [6]:
items = pd.read_csv('items_catalog.csv')

In [7]:
print(items.columns)
print(items.dtypes)

Index(['item_id', 'brand'], dtype='object')
item_id     int64
brand      object
dtype: object


In [8]:
len(items['brand'].unique())

274

Items table will be joined to interactions so it has to be checked for 1-1 mapping

In [9]:
max(items.groupby('item_id')['brand'].nunique())

3

In [10]:
items.groupby('item_id').agg({"brand": list}).head(30)
#there are branded with not set brands or not found brands, should always prefer brand over no brand, might be due to error

,brand
item_id,
3752,[Sohi]
3762,[Sohi]
3763,"[Sohi, (not set)]"
3771,[(not set)]
3773,[V.yher]
3774,"[Sohi, (not set)]"
3776,"[Sohi, (not set)]"
3783,[V.yher]
3788,[V.yher]


In [11]:
# Convert (not set) to np.nan
items['brand'] = items['brand'].apply(lambda x: np.nan if x == '(not set)' else x)

In [12]:
# Split into two sets one branded and one unbranded
items1 = items[items.brand.notnull()] #branded
items2 = items[(items.brand.isnull()) & (~(items.item_id).isin(list(items1.item_id)))] #unbranded conditional on the fact that items which are branded will not show in unbranded
items = pd.concat([items1, items2], axis = 0)
items['brand'] = items['brand'].fillna('(not set)')

In [13]:
print(items.groupby('item_id').count().mean()) 
sum(items.groupby('item_id').count()['brand']>1) #how many are more than one brand
# result checked

brand    1.00044
dtype: float64


24

## Join dataframes

In [14]:
#ID dtypes match so just merge no need to convert
df = interactions.merge(items, on = 'item_id', how = 'left')

## Convert to proper data types

In [15]:
df['interaction_time'] = pd.to_datetime(df['interaction_time'])

## Simple EDA

In [16]:
df.interaction_type.value_counts()
#remove cart may indicate a churn too, instead of just inactivity

ProductView       9660168
AddToCart         5770244
RemoveFromCart    3981236
Purchase          1287291
Name: interaction_type, dtype: int64

In [17]:
print(df.columns)

Index(['interaction_time', 'user_id', 'item_id', 'interaction_type', 'price',
       'brand'],
      dtype='object')


In [18]:
df.describe()

,item_id,price
count,2.069894e+07,2.069894e+07
mean,5.484067e+06,8.534317e+00
std,1.306184e+06,1.937877e+01
min,3.752000e+03,-7.937000e+01
25%,5.724650e+06,2.060000e+00
50%,5.810720e+06,4.050000e+00
75%,5.857866e+06,7.030000e+00
max,5.932595e+06,3.277800e+02


# First and Last visit
Assume interaction of all types

In [19]:
df.sort_values(by = ['user_id', 'interaction_time'], inplace = True)
df

,interaction_time,user_id,item_id,interaction_type,price,brand
10413399,2019-12-16 09:15:42,00000a88-2beb-4f3a-bf8a,5875317,AddToCart,1.59,(not set)
10413523,2019-12-16 09:16:37,00000a88-2beb-4f3a-bf8a,6941,ProductView,4.71,Mykek
10413733,2019-12-16 09:17:56,00000a88-2beb-4f3a-bf8a,5875317,ProductView,1.59,(not set)
10730399,2019-12-18 15:24:48,00000a88-2beb-4f3a-bf8a,5560760,ProductView,201.59,Jixo
10959006,2019-12-20 15:22:20,00000a88-2beb-4f3a-bf8a,5560760,ProductView,201.59,Jixo
...,...,...,...,...,...,...
12363432,2019-11-04 11:38:44,ffffe471-172d-42f1-af9d,5833695,ProductView,7.22,(not set)
2554144,2020-02-18 09:54:38,ffffe471-172d-42f1-af9d,5882610,ProductView,5.56,Zowu
8129967,2020-01-29 20:10:14,ffffeb30-148d-4c1a-a73c,5693515,ProductView,5.56,(not set)
3339806,2020-02-24 11:59:46,fffff28a-568b-4cdc-a4ef,5859177,ProductView,2.19,(not set)


In [20]:
visits = df.groupby(['user_id']).agg({'interaction_time': [min, max]}).reset_index()

In [21]:
visits.columns = ['user_id', 'min', 'max']
visits

,user_id,min,max
0,00000a88-2beb-4f3a-bf8a,2019-12-16 09:15:42,2020-02-26 12:59:50
1,00000cfa-9a75-4ee4-8ff0,2020-02-07 15:53:22,2020-02-29 04:19:54
2,00001957-83e2-4e36-aacf,2020-01-31 07:07:19,2020-02-11 15:06:17
3,00001af7-43f5-4c04-be0e,2020-02-13 18:08:22,2020-02-20 16:16:13
4,0000200c-8a05-44d7-bbd6,2019-11-29 12:45:04,2019-11-29 12:45:04
...,...,...,...
1639353,ffffbee2-917e-48cd-a532,2020-02-02 08:32:45,2020-02-02 08:32:45
1639354,ffffe471-172d-42f1-af9d,2019-11-04 11:38:44,2020-02-18 09:54:38
1639355,ffffeb30-148d-4c1a-a73c,2020-01-29 20:10:14,2020-01-29 20:10:14
1639356,fffff28a-568b-4cdc-a4ef,2020-02-24 11:59:46,2020-02-24 11:59:46


# Average Time Between Two Visits
- Assume any kind of interaction given that it happens on a different date
- Assume any kind of interaction no condition

In [22]:
df['date'] = df['interaction_time'].dt.date
tbt = df.groupby(['user_id', 'date']).agg({'interaction_time':[min, max]}).reset_index()
tbt.columns = ['user_id', 'date', 'interaction_min', 'interaction_max']
tbt['next_interaction'] = tbt.groupby(['user_id'])['interaction_max'].shift(-1)
tbt['next_visit_interval'] = tbt['next_interaction'] - tbt['interaction_min']
tbt['next_visit_interval'] = tbt['next_visit_interval'].apply(lambda x: x.seconds/3600)
tbt = tbt.groupby(['user_id'])['next_visit_interval'].mean().reset_index()

In [23]:
df['next_interaction_time'] = df.groupby(['user_id'])['interaction_time'].shift(-1)
df['next_visit_interval'] = df['next_interaction_time'] - df['interaction_time']
df['next_visit_interval'] = df['next_visit_interval'].apply(lambda x: x.seconds/3600)
tbt_2 = df.groupby(['user_id'])['next_visit_interval'].mean().reset_index()

In [24]:
time_bwtn = tbt.merge(tbt_2, on = 'user_id')
time_bwtn

,user_id,next_visit_interval_x,next_visit_interval_y
0,00000a88-2beb-4f3a-bf8a,7.500069,5.155648
1,00000cfa-9a75-4ee4-8ff0,12.442222,6.221111
2,00001957-83e2-4e36-aacf,14.299306,2.332616
3,00001af7-43f5-4c04-be0e,22.130833,0.962210
4,0000200c-8a05-44d7-bbd6,NaN,NaN
...,...,...,...
1639353,ffffbee2-917e-48cd-a532,NaN,NaN
1639354,ffffe471-172d-42f1-af9d,22.265000,22.265000
1639355,ffffeb30-148d-4c1a-a73c,NaN,NaN
1639356,fffff28a-568b-4cdc-a4ef,NaN,NaN


# Average Monthly Spending
Take total spending of a user since the first visit to the most current record found

In [25]:
purchases = df[df['interaction_type'] == 'Purchase']
purchases

,interaction_time,user_id,item_id,interaction_type,price,brand,date,next_interaction_time,next_visit_interval
1589253,2020-02-11 12:47:56,000148b5-6ec2-47a5-83ba,5836495,Purchase,4.76,Lymyso,2020-02-11,2020-02-11 12:47:56,0.000000
1589254,2020-02-11 12:47:56,000148b5-6ec2-47a5-83ba,5836500,Purchase,4.76,Lymyso,2020-02-11,2020-02-11 12:47:56,0.000000
1589255,2020-02-11 12:47:56,000148b5-6ec2-47a5-83ba,5836501,Purchase,4.76,Lymyso,2020-02-11,2020-02-11 12:47:56,0.000000
1589256,2020-02-11 12:47:56,000148b5-6ec2-47a5-83ba,5836502,Purchase,4.76,Lymyso,2020-02-11,2020-02-11 12:47:56,0.000000
1589257,2020-02-11 12:47:56,000148b5-6ec2-47a5-83ba,5836503,Purchase,4.76,Lymyso,2020-02-11,2020-02-11 12:47:56,0.000000
...,...,...,...,...,...,...,...,...,...
5299951,2020-01-10 17:52:00,ffff6bd0-f885-48f1-b613,5584836,Purchase,4.95,Nu.jahi,2020-01-10,2020-01-10 17:52:00,0.000000
5299953,2020-01-10 17:52:00,ffff6bd0-f885-48f1-b613,5584838,Purchase,4.95,Nu.jahi,2020-01-10,2020-01-10 17:52:00,0.000000
5299955,2020-01-10 17:52:00,ffff6bd0-f885-48f1-b613,5834736,Purchase,3.02,(not set),2020-01-10,2020-01-10 17:52:00,0.000000
5299956,2020-01-10 17:52:00,ffff6bd0-f885-48f1-b613,5846442,Purchase,19.52,Tydukipa,2020-01-10,2020-01-10 17:52:00,0.000000


In [26]:
latest_date = df['interaction_time'].max()
latest_date

Timestamp('2020-02-29 23:59:59')

In [27]:
visits['months_since_first'] = visits["min"].apply(lambda x: (latest_date.year - x.year)*12 - latest_date.month - x.month+1)

In [28]:
monthly = purchases.groupby(['user_id'])['price'].sum().reset_index()

In [29]:
monthly = monthly.merge(visits, on = 'user_id', how = 'left')

In [30]:
monthly['avg_user'] = monthly['price']/monthly['months_since_first']
monthly

,user_id,price,min,max,months_since_first,avg_user
0,000148b5-6ec2-47a5-83ba,64.26,2020-02-11 00:02:27,2020-02-29 22:00:56,-3,-21.42
1,0002c237-9155-4d19-a0b4,133.54,2019-12-01 11:42:37,2019-12-05 16:47:20,-1,-133.54
2,0003fc81-ef96-4d4a-a567,10.14,2019-12-02 11:32:54,2019-12-02 13:00:21,-1,-10.14
3,00056859-dca9-4a63-af7c,66.56,2020-01-20 14:55:51,2020-02-28 14:06:06,-2,-33.28
4,00059bbf-0b42-43a4-ad77,14.60,2020-01-10 20:37:13,2020-01-11 04:38:34,-2,-7.30
...,...,...,...,...,...,...
110513,fffca46c-4121-48de-bc62,12.64,2019-10-02 12:27:48,2019-10-03 16:21:15,1,12.64
110514,fffda61e-f290-4f8e-b4f5,4.76,2020-01-09 00:05:14,2020-01-09 00:15:52,-2,-2.38
110515,fffe9def-a623-41fd-bda4,64.60,2020-01-03 20:43:08,2020-01-03 20:52:59,-2,-32.30
110516,fffebbf0-b111-47a5-b45f,37.61,2019-10-01 13:41:30,2019-12-07 16:45:12,1,37.61


# User Favorite Brand
- Most frequent purchases from brand by user if no purchase then most frequent product view, since purchase is worth more than view 

In [31]:
fav_purchase = df[df['interaction_type'] == 'Purchase'].groupby(['user_id', 'brand'])['interaction_type'].count().reset_index().sort_values(by = ['user_id', 'interaction_type'])
fav_view = df[df['interaction_type'] == 'ProductView'].groupby(['user_id', 'brand'])['interaction_type'].count().reset_index().sort_values(by = ['user_id', 'interaction_type'])


In [32]:
fav_purchase

,user_id,brand,interaction_type
0,000148b5-6ec2-47a5-83ba,Lymyso,17
3,0002c237-9155-4d19-a0b4,Lymyso,1
4,0002c237-9155-4d19-a0b4,Rayul,1
6,0002c237-9155-4d19-a0b4,Wuhohuci,1
2,0002c237-9155-4d19-a0b4,Kewukeqam,2
...,...,...,...
410761,fffebbf0-b111-47a5-b45f,Yuzusaxay,1
410760,fffebbf0-b111-47a5-b45f,Sokej,3
410762,ffff6bd0-f885-48f1-b613,(not set),1
410763,ffff6bd0-f885-48f1-b613,Nu.jahi,2


In [217]:
fav_purchase = fav_purchase.groupby('user_id')['brand'].last().reset_index()

In [220]:
fav_purchase.shape

(110518, 2)

In [218]:
fav_view = fav_view.groupby('user_id')['brand'].last().reset_index()

In [221]:
fav_view.shape

(1597754, 2)

In [228]:
fav_brand = fav_view.merge(fav_purchase, on = 'user_id', how = 'outer')
fav_brand

,user_id,brand_x,brand_y
0,00000a88-2beb-4f3a-bf8a,Mykek,NaN
1,00000cfa-9a75-4ee4-8ff0,Beb ip aq,NaN
2,00001957-83e2-4e36-aacf,(not set),NaN
3,00001af7-43f5-4c04-be0e,(not set),NaN
4,0000200c-8a05-44d7-bbd6,Kyyo,NaN
...,...,...,...
1603061,ffdd5b01-6d43-4194-9c3e,NaN,(not set)
1603062,ffdd719c-6736-4c4e-8909,NaN,(not set)
1603063,ffe1e12b-c176-4671-855e,NaN,(not set)
1603064,fff33221-4550-47dd-b4e9,NaN,Tezuwiv


In [229]:
sum(fav_brand['brand_y'].notnull())

110518

In [230]:
fav_brand['fav_brand'] = fav_brand['brand_y'].combine_first(fav_brand['brand_x'])
fav_brand

,user_id,brand_x,brand_y,fav_brand
0,00000a88-2beb-4f3a-bf8a,Mykek,NaN,Mykek
1,00000cfa-9a75-4ee4-8ff0,Beb ip aq,NaN,Beb ip aq
2,00001957-83e2-4e36-aacf,(not set),NaN,(not set)
3,00001af7-43f5-4c04-be0e,(not set),NaN,(not set)
4,0000200c-8a05-44d7-bbd6,Kyyo,NaN,Kyyo
...,...,...,...,...
1603061,ffdd5b01-6d43-4194-9c3e,NaN,(not set),(not set)
1603062,ffdd719c-6736-4c4e-8909,NaN,(not set),(not set)
1603063,ffe1e12b-c176-4671-855e,NaN,(not set),(not set)
1603064,fff33221-4550-47dd-b4e9,NaN,Tezuwiv,Tezuwiv


# explore churn
one thing that jumps out to me is the fact that there is remove from cart, what if i count the number of add to cart to remove from cart, not related to the link provided

- churn should be more item and brand based 
- inactivity over certain periods of time too maybe

In [234]:
churn = df.groupby(['user_id', 'interaction_type']).count().reset_index() 
new = churn.pivot(index='user_id', columns='interaction_type', values='item_id')

In [235]:
new

interaction_type,AddToCart,ProductView,Purchase,RemoveFromCart
user_id,,,,
00000a88-2beb-4f3a-bf8a,3.0,19.0,NaN,3.0
00000cfa-9a75-4ee4-8ff0,NaN,3.0,NaN,NaN
00001957-83e2-4e36-aacf,6.0,15.0,NaN,4.0
00001af7-43f5-4c04-be0e,11.0,4.0,NaN,9.0
0000200c-8a05-44d7-bbd6,NaN,1.0,NaN,NaN
...,...,...,...,...
ffffbee2-917e-48cd-a532,NaN,1.0,NaN,NaN
ffffe471-172d-42f1-af9d,NaN,2.0,NaN,NaN
ffffeb30-148d-4c1a-a73c,NaN,1.0,NaN,NaN


In [241]:
(new['RemoveFromCart']/new['AddToCart']).mean()

0.7876480857292346

In [240]:
(new['Purchase']/new['AddToCart']).mean()

0.6253147498382713

???? not proportional